In [4]:
#Installer les librairies nécessaire 
pip install numpy scikit-image Pillow pandas tqdm


Note: you may need to restart the kernel to use updated packages.


In [13]:
#importation des librairies
import numpy as np
from skimage.draw import polygon2mask
from PIL import Image
import os
import json
import pandas as pd
from tqdm import tqdm

In [16]:
#creer un programme qui parcours l'image et son label fichier json afin de lui creer un masque en .png 